In [ ]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np

In [ ]:
# load and prepare data
competence = pickle.load(open( '../00_data/final/competence.pkl','rb') )
dropout = pickle.load(open( '../00_data/final/dropout_per_session.pkl','rb') )
mistakes = pickle.load(open( '../00_data/final/incorrect_answers.pkl','rb') )
xmlsaetze = pickle.load(open( '../00_data/cleaned/xmlsaetze.pkl','rb') )
sitzungen = pickle.load(open( '../00_data/cleaned/sitzungssummary.pkl','rb') )
saetze = pickle.load(open( '../00_data/cleaned/saetze.pkl','rb') )
xmlsaetze = pd.merge(xmlsaetze, sitzungen, on='UserID', how='left')

dropout = dropout.drop(columns=['Geschlecht','Klassenstufe','interventiongroup'])
dropout = dropout[['UserID','dropout']]
dropout = dropout.drop_duplicates()

In [ ]:
## Corr between mistakes and dropout
mistakes_dropout = pd.merge(mistakes, dropout, on='UserID', how='inner')
mistakes_dropout = mistakes_dropout[['interventiongroup','prop_fehler','dropout']]

intvcontrol = mistakes_dropout[(mistakes_dropout['interventiongroup']=='control')]
intv2 = mistakes_dropout[(mistakes_dropout['interventiongroup']=='2')]
intv3 = mistakes_dropout[(mistakes_dropout['interventiongroup']=='3')]
intv4 = mistakes_dropout[(mistakes_dropout['interventiongroup']=='4')]
intv5 = mistakes_dropout[(mistakes_dropout['interventiongroup']=='5')]
intv6 = mistakes_dropout[(mistakes_dropout['interventiongroup']=='6')]

li = [mistakes_dropout, intvcontrol, intv2, intv3, intv4, intv5, intv6]
for x in li:
    print(np.corrcoef(x.dropout, x.prop_fehler))
    print('--------------------------')

In [ ]:
## Corr between competence and dropout
competence_= competence.drop(columns=['Geschlecht','Klassenstufe'])
competence_dropout = pd.merge(competence_, dropout, on='UserID', how='inner')
competence_dropout = competence_dropout[['interventiongroup','ability','dropout']]

intvcontrol = competence_dropout[(competence_dropout['interventiongroup']=='control')]
intv2 = competence_dropout[(competence_dropout['interventiongroup']=='2')]
intv3 = competence_dropout[(competence_dropout['interventiongroup']=='3')]
intv4 = competence_dropout[(competence_dropout['interventiongroup']=='4')]
intv5 = competence_dropout[(competence_dropout['interventiongroup']=='5')]
intv6 = competence_dropout[(competence_dropout['interventiongroup']=='6')]

li = [competence_dropout, intvcontrol, intv2, intv3, intv4, intv5, intv6]
for x in li:
    print(np.corrcoef(x.dropout, x.ability))
    print('--------------------------')

In [ ]:
## Corr between mistakes und competence
competence_mistakes = competence.drop(columns=['Geschlecht','Klassenstufe','interventiongroup'])
mistakes_competence = pd.merge(mistakes, competence_mistakes, on='UserID', how='inner')
mistakes_competence = mistakes_competence[['interventiongroup','prop_fehler','ability']]

intvcontrol = mistakes_competence[(mistakes_competence['interventiongroup']=='control')]
intv2 = mistakes_competence[(mistakes_competence['interventiongroup']=='2')]
intv3 = mistakes_competence[(mistakes_competence['interventiongroup']=='3')]
intv4 = mistakes_competence[(mistakes_competence['interventiongroup']=='4')]
intv5 = mistakes_competence[(mistakes_competence['interventiongroup']=='5')]
intv6 = mistakes_competence[(mistakes_competence['interventiongroup']=='6')]

li = [mistakes_competence, intvcontrol, intv2, intv3, intv4, intv5, intv6]
for x in li:
    print(np.corrcoef(x.ability, x.prop_fehler))
    print('--------------------------')

In [ ]:
## Count of sentence per user and intv group
cnt_saetze_pro_user = xmlsaetze.groupby(['UserID','interventiongroup']).agg({'ID':'count'}).reset_index()
final = cnt_saetze_pro_user[['ID','interventiongroup']]
print(sns.boxplot(x=final['ID'],y=final["interventiongroup"],showfliers = False))

In [ ]:
## Corr count of sentences and competence
corr_ = pd.merge(cnt_saetze_pro_user, competence_, on='UserID')
print(np.corrcoef(corr_.ID, corr_.ability))

In [ ]:
## Mean difficulty by interventiongroup
saetze = saetze[['satzID','Schwierigkeit']]
xmlsaetze = pd.merge(xmlsaetze, saetze, left_on='SatzID', right_on='satzID',how='left')
xmlsaetze = pd.merge(xmlsaetze, sitzungen, on='UserID', how='left')

diffic_pro_user = xmlsaetze.groupby(['UserID','interventiongroup']).agg({'Schwierigkeit':'mean'}).reset_index()
print(sns.boxplot(x=diffic_pro_user['Schwierigkeit'],y=diffic_pro_user["interventiongroup"],showfliers = False))

In [ ]:
## Corr between difficulty and competence
corr_2 = pd.merge(diffic_pro_user, competence_, on='UserID')
print(np.corrcoef(corr_2.Schwierigkeit, corr_2.ability))